ใน Training Loop หรือฟังก์ชัน `fit()` จะมี Parameter ที่ใช้ในการเทรน ดังนี้

> `fit(epoch, model, loss_func, opt, train_dl, valid_dl)`

โดยขั้นแรก เนื่องจาก train_dl และ valid_dl คือ 2 Object ของ Class เดียวกัน ข้อมูลข้างในก็มาจากที่เดียวกัน เราจะ Refactor สร้าง Class ใหม่ ชื่อว่า [DataBunch](#3.-DataBunch) มาเป็น Wrapper Class ไว้ จะได้มองเป็น Unit เดียวกัน เวลาจัดการก็จัดการด้วยวิธีคล้าย ๆ กัน พร้อม ๆ กัน เพื่อให้โค้ด Clean มากขึ้น

จะได้ Training Loop ใหม่ เป็น

> `fit(epoch, model, loss_func, opt, databunch)`

# 0. Magic

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Import

In [0]:
import torch
from torch import tensor
from torch.nn import *
import torch.nn.functional as F
from torch.utils.data import *
from fastai import datasets
from fastai.metrics import accuracy
import pickle, gzip, math, torch

# 2. Data

In [0]:
class Dataset(data.Dataset):
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [0]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

In [0]:
def normalize(x, m, s): 
    return (x-m)/s

In [0]:
train_mean, train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [0]:
nh, bs = 100, 32
n, m = x_train.shape
c = (y_train.max()+1).numpy()
loss_func = F.cross_entropy

In [0]:
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(valid_ds, bs)

# 3. DataBunch

สร้าง Class DataBunch สำหรับ Wrap Training Set DataLoader และ Validation Set DataLoader 

และใส่ จำนวน Class ของ Output ไว้ด้วย จะได้ใช้ตอนสร้างโมเดล

In [0]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

ลองสร้าง DataBunch จาก train_dl, valid_dl และ c ที่เราสร้างไว้ก่อนหน้านี้

In [0]:
databunch = DataBunch(train_dl, valid_dl, c)

In [0]:
lr = 0.03
epoch = 10
nh = 50

In [0]:
def get_model():
    # loss function
    loss_func = F.cross_entropy
    model = Sequential(Linear(m, nh), ReLU(), Linear(nh,c))
    return model, loss_func

In [0]:
model, loss_func = get_model()
optim = torch.optim.SGD(model.parameters(), lr=lr)

เราจะได้ Training Loop ฟังก์ชัน fit ที่โค้ด Clean มากขึ้น

In [0]:
def fit(epoch, model, loss_func, optim, databunch):
    # e = epoch number
    for e in range(epoch):

        # Set Model in Train Mode
        model.train()

        for xb, yb in databunch.train_dl:
            yhatb = model(xb)
            loss = loss_func(yhatb, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()

        # Set Model in Evaluation Mode
        model.eval()

        # Metrics
        with torch.no_grad():
            # tot_loss = total loss, tot_acc = total accuracy
            tot_loss, tot_acc = 0., 0.
            for xb, yb in databunch.valid_dl:
                yhatb = model(xb)
                tot_acc += accuracy(yhatb, yb)
                tot_loss += loss_func(yhatb, yb)
            # nv = number of validation batch
            nv = len(valid_ds)/bs
            print(f'epoch={e}, valid_loss={tot_loss/nv}, valid_acc={tot_acc/nv}')            
    return tot_loss/nv, tot_acc/nv
    
    

ลองรัน fit ให้เทรนโมเดลดู

In [0]:
fit(epoch, model, loss_func, optim, databunch)

epoch=0, valid_loss=0.21391521394252777, valid_acc=0.9406999945640564
epoch=1, valid_loss=0.1618962585926056, valid_acc=0.9544000029563904
epoch=2, valid_loss=0.13810059428215027, valid_acc=0.9596999883651733
epoch=3, valid_loss=0.12372233718633652, valid_acc=0.9642000198364258
epoch=4, valid_loss=0.11448055505752563, valid_acc=0.9668999910354614
epoch=5, valid_loss=0.10786031186580658, valid_acc=0.9699000120162964
epoch=6, valid_loss=0.10382169485092163, valid_acc=0.9717000126838684
epoch=7, valid_loss=0.10144499689340591, valid_acc=0.9724000096321106
epoch=8, valid_loss=0.099140465259552, valid_acc=0.9733999967575073
epoch=9, valid_loss=0.09910203516483307, valid_acc=0.9732000231742859


(tensor(0.0991), tensor(0.9732))

# 4. Learner

ใน Training Loop หรือฟังก์ชัน `fit()` จะเหลือ Parameter ที่ใช้ในการเทรน ดังนี้

> `fit(epoch, model, loss_func, optim, databunch)`

ขั้นต่อมา เนื่องจาก model, loss_func, optim และ databunch จะเป็นสิ่งที่ทำงานร่วมกันตลอด เราจะ Refactor สร้าง Class ใหม่ ชื่อว่า Learner มาเป็น Wrapper Class ไว้ จะได้มองเป็น Unit เดียวกัน เวลาจัดการก็จัดการด้วยวิธีคล้าย ๆ กัน พร้อม ๆ กัน เพื่อให้โค้ด Clean มากขึ้น

จะได้ Training Loop ใหม่ เป็น

> `fit(epoch, learner)`

In [0]:
class Learner():
    def __init__(self, model, optim, loss_func, databunch):
        self.model, self.optim, self.loss_func, self.databunch = model, optim, loss_func, databunch


สร้าง Learner ขึ้นมาจาก model, optim, loss_func และ databunch ที่เราเตรียมไว้ก่อนหน้า 

In [0]:
learner = Learner(model, optim, loss_func, databunch)

เราจะได้ fit เวอร์ชัน 2 ที่ Clean ขึ้น รับ Parameter แค่ 2 ตัว

In [0]:
def fit2(epoch, learner):
    # e = epoch number
    for e in range(epoch):

        # Set Model in Train Mode
        learner.model.train()

        for xb, yb in learner.databunch.train_dl:
            yhatb = learner.model(xb)
            loss = learner.loss_func(yhatb, yb)
            loss.backward()
            learner.optim.step()
            learner.optim.zero_grad()

        # Set Model in Evaluation Mode
        learner.model.eval()

        # Metrics
        with torch.no_grad():
            # tot_loss = total loss, tot_acc = total accuracy
            tot_loss, tot_acc = 0., 0.
            for xb, yb in learner.databunch.valid_dl:
                yhatb = learner.model(xb)
                tot_acc += accuracy(yhatb, yb)
                tot_loss += learner.loss_func(yhatb, yb)
            # nv = number of validation batch
            nv = len(learner.databunch.valid_ds)/bs
            print(f'epoch={e}, valid_loss={tot_loss/nv}, valid_acc={tot_acc/nv}')            
    return tot_loss/nv, tot_acc/nv
    
    

ลองรัน fit เวอร์ชัน 2

In [0]:
fit2(epoch, learner)

epoch=0, valid_loss=0.09939024597406387, valid_acc=0.9733999967575073
epoch=1, valid_loss=0.10003691166639328, valid_acc=0.9739999771118164
epoch=2, valid_loss=0.10111748427152634, valid_acc=0.9745000004768372
epoch=3, valid_loss=0.10160576552152634, valid_acc=0.9745000004768372
epoch=4, valid_loss=0.10229545086622238, valid_acc=0.9746999740600586
epoch=5, valid_loss=0.10302049666643143, valid_acc=0.9745000004768372
epoch=6, valid_loss=0.10348284989595413, valid_acc=0.9747999906539917
epoch=7, valid_loss=0.10397198796272278, valid_acc=0.9749000072479248
epoch=8, valid_loss=0.10495433211326599, valid_acc=0.9750000238418579
epoch=9, valid_loss=0.10624222457408905, valid_acc=0.9750000238418579


(tensor(0.1062), tensor(0.9750))

# Credit

* https://course.fast.ai/videos/?lesson=9
* http://yann.lecun.com/exdb/mnist/